In [1]:
# Imports for all tasks
import torch
import torch.nn as nn
from torch.optim import Adam
from sentence_transformers import SentenceTransformer

print("Setup complete. Required libraries imported.")

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setup complete. Required libraries imported.


Task 1

In [7]:
from sentence_transformers import SentenceTransformer

# Task 1: Collect sentences
sentences_task1 = []
print("Enter sentences to generate embeddings (type 'done' when finished):")

while True:
    sentence = input("> ")
    if sentence.lower() == 'done':
        break
    sentences_task1.append(sentence)

if not sentences_task1:
    print("No sentences provided.")
else:
    # Generate and display embeddings
    model_task1 = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model_task1.encode(sentences_task1, batch_size=32, show_progress_bar=False)

    print("\nGenerated Sentence Embeddings:")
    print("-----------------------------")
    for i, (sentence, embedding) in enumerate(zip(sentences_task1, embeddings), 1):
        print(f"Sentence {i}: {sentence}")
        print(f"Embedding (first 5 of 384 dimensions): {embedding[:5]}...\n")


Enter sentences to generate embeddings (type 'done' when finished):
> apples 5 pcs 7$
> milk 2 pcs 12$
> bread 1 pcs 6$
> done

Generated Sentence Embeddings:
-----------------------------
Sentence 1: apples 5 pcs 7$
Embedding (first 5 of 384 dimensions): [-0.01811078  0.04169159  0.06918947  0.02195263  0.01449898]...

Sentence 2: milk 2 pcs 12$
Embedding (first 5 of 384 dimensions): [ 0.03493364  0.00117    -0.00209289 -0.01365882 -0.00412777]...

Sentence 3: bread 1 pcs 6$
Embedding (first 5 of 384 dimensions): [-0.00151778  0.02710414 -0.01061682  0.00802708 -0.07450908]...



Task 2

In [3]:
# Task 2: Define the MTL model
class MultiTaskModel(nn.Module):
    def __init__(self, num_categories=5):
        super(MultiTaskModel, self).__init__()
        self.backbone = SentenceTransformer('all-MiniLM-L6-v2')
        self.embedding_dim = 384
        self.classification_head = nn.Linear(self.embedding_dim, num_categories)
        self.regression_head = nn.Linear(self.embedding_dim, 1)
    
    def forward(self, sentences):
        embeddings = self.backbone.encode(sentences, convert_to_tensor=True, 
                                        batch_size=32, show_progress_bar=False)
        embeddings = embeddings.to(self.classification_head.weight.device)
        class_logits = self.classification_head(embeddings)
        quantity_pred = self.regression_head(embeddings)
        return class_logits, quantity_pred

# Initialize model
model_task2 = MultiTaskModel(num_categories=5)
category_labels = ["Fruit", "Dairy", "Bakery", "Meat", "Other"]
print("MTL model initialized.")

MTL model initialized.


In [4]:
# Task 2: Hardcoded sentences and predictions
sentences_task2 = [
    "2x Apples $1.99", "Milk 1L $2.50", "3x Bread $3.00",
    "Chicken Breast 500g $5.99", "4x Bananas $2.40"
]

print("Processing the following receipt items:")
for i, sentence in enumerate(sentences_task2, 1):
    print(f"{i}. {sentence}")

# Get predictions
model_task2.eval()
with torch.no_grad():
    class_logits, quantity_pred = model_task2(sentences_task2)
    class_probs = torch.softmax(class_logits, dim=1)
    class_indices = torch.argmax(class_probs, dim=1)
    
    print("\nMulti-Task Predictions:")
    print("----------------------")
    for i, (sentence, category_idx, qty) in enumerate(zip(sentences_task2, class_indices, quantity_pred)):
        category = category_labels[category_idx.item()]
        quantity = qty.item()
        print(f"Sentence {i+1}: {sentence}")
        print(f"Predicted Category: {category}")
        print(f"Predicted Quantity: {quantity:.2f}\n")

Processing the following receipt items:
1. 2x Apples $1.99
2. Milk 1L $2.50
3. 3x Bread $3.00
4. Chicken Breast 500g $5.99
5. 4x Bananas $2.40

Multi-Task Predictions:
----------------------
Sentence 1: 2x Apples $1.99
Predicted Category: Other
Predicted Quantity: -0.02

Sentence 2: Milk 1L $2.50
Predicted Category: Other
Predicted Quantity: -0.01

Sentence 3: 3x Bread $3.00
Predicted Category: Dairy
Predicted Quantity: -0.00

Sentence 4: Chicken Breast 500g $5.99
Predicted Category: Other
Predicted Quantity: -0.02

Sentence 5: 4x Bananas $2.40
Predicted Category: Other
Predicted Quantity: -0.06



Task 3

Task 4

In [10]:
# Hypothetical Sentences
sentences = ["I love ML!", "This is boring.", "Transformers rock!"]

# Hypothetical Labels (Task A: Tech vs Non-Tech, Task B: Sentiment)
labels_a = torch.tensor([1, 0, 1], dtype=torch.long)  # Tech: 1, Non-Tech: 0
labels_b = torch.tensor([1, 0, 1], dtype=torch.long)  # Positive: 1, Negative: 0

# Encode Sentences to Numeric Embeddings
encoder = SentenceTransformer('all-MiniLM-L6-v2')
sentence_embeddings = torch.tensor(encoder.encode(sentences))

# Define Multi-Task Model (Assuming it has two outputs)
class MultiTaskModel(nn.Module):
    def __init__(self, num_categories=2):
        super(MultiTaskModel, self).__init__()
        self.shared_layer = nn.Linear(384, 128)  # 384-d input (MiniLM)
        self.task_head_a = nn.Linear(128, num_categories)  # Task A Head
        self.task_head_b = nn.Linear(128, num_categories)  # Task B Head

    def forward(self, x):
        shared_output = torch.relu(self.shared_layer(x))
        logits_a = self.task_head_a(shared_output)  # Tech/Non-Tech
        logits_b = self.task_head_b(shared_output)  # Sentiment
        return logits_a, logits_b

# Initialize Model, Optimizer, and Loss Function
model = MultiTaskModel(num_categories=2)
optimizer = Adam(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

# Training Loop (3 Epochs for Demonstration)
for epoch in range(3):
    model.train()
    optimizer.zero_grad()

    # Forward Pass
    logits_a, logits_b = model(sentence_embeddings)

    # Compute Loss for Each Task
    loss_a = loss_fn(logits_a, labels_a)
    loss_b = loss_fn(logits_b, labels_b)
    total_loss = loss_a + loss_b  # Combined loss

    # Backward Pass
    total_loss.backward()
    optimizer.step()

    # Compute Predictions
    preds_a = torch.argmax(logits_a, dim=1)
    preds_b = torch.argmax(logits_b, dim=1)

    # Compute Accuracy for Each Task
    acc_a = (preds_a == labels_a).float().mean().item()
    acc_b = (preds_b == labels_b).float().mean().item()

    # Print Epoch Summary
    print(f"Epoch {epoch+1}/3")
    print(f"Loss A: {loss_a.item():.4f}, Loss B: {loss_b.item():.4f}, Total Loss: {total_loss.item():.4f}")
    print(f"Accuracy A: {acc_a:.4f}, Accuracy B: {acc_b:.4f}\n")


Epoch 1/3
Loss A: 0.6974, Loss B: 0.6898, Total Loss: 1.3873
Accuracy A: 0.3333, Accuracy B: 0.6667

Epoch 2/3
Loss A: 0.6972, Loss B: 0.6895, Total Loss: 1.3866
Accuracy A: 0.3333, Accuracy B: 0.6667

Epoch 3/3
Loss A: 0.6969, Loss B: 0.6891, Total Loss: 1.3860
Accuracy A: 0.3333, Accuracy B: 1.0000

